# CNN Training with Fashion Data set - Pytorch A - Z

In [6]:
import torch
import torch.nn as nn 
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import torchvision.transforms as tranforms

torch.set_printoptions(linewidth=120)  # Displa optin for output
torch.set_grad_enabled(True) # Already on by default

In [7]:
print(torch.__version__)
print(torchvision.__version__)

1.2.0
0.4.0


In [8]:
def get_num_correct(preds, labels):
    return preds.argmax(dim=1).eq(labels).sum().item()

## Load data set 

In [9]:
train_set = torchvision.datasets.FashionMNIST(
    root='./data'
    ,train=True
    ,download=True
    ,transform=tranforms.Compose([
        tranforms.ToTensor()
    ])
)

train_loader = torch.utils.data.DataLoader(
    train_set
    ,batch_size=64
    ,shuffle=True
)

test_set = torchvision.datasets.FashionMNIST(
    root='./data'
    ,train=False
    ,download=True
    ,transform=tranforms.Compose([
        tranforms.ToTensor()
    ])
)

test_loader = torch.utils.data.DataLoader(
    test_set
    ,batch_size=64
    ,shuffle=True
)



0it [00:00, ?it/s]

  0%|                                                                                     | 0/26421880 [00:00<?, ?it/s]

  0%|                                                                      | 16384/26421880 [00:00<05:31, 79599.36it/s]

  0%|▏                                                                     | 49152/26421880 [00:00<04:31, 97105.34it/s]

  0%|▎                                                                   | 106496/26421880 [00:01<03:30, 124904.74it/s]

  1%|▌                                                                   | 221184/26421880 [00:01<02:36, 167828.80it/s]

  1%|▊                                                                   | 294912/26421880 [00:01<02:01, 215831.41it/s]

  1%|▉                                                                   | 385024/26421880 [00:01<01:33, 279041.49it/s]

  2%|█▎                                                                  | 499712/26421880 [00:01<01:11, 360066.96it/s]

  2%|█▋   

 35%|███████████████████████▌                                           | 9289728/26421880 [00:08<00:18, 903920.10it/s]

 36%|███████████████████████▊                                           | 9396224/26421880 [00:08<00:18, 931323.50it/s]

 36%|████████████████████████                                           | 9494528/26421880 [00:09<00:18, 893867.92it/s]

 36%|████████████████████████▎                                          | 9601024/26421880 [00:09<00:18, 931513.08it/s]

 37%|████████████████████████▌                                          | 9699328/26421880 [00:09<00:19, 848569.10it/s]

 37%|████████████████████████▊                                          | 9805824/26421880 [00:09<00:18, 884226.91it/s]

 38%|█████████████████████████▏                                         | 9920512/26421880 [00:09<00:17, 940629.56it/s]

 38%|█████████████████████████                                         | 10027008/26421880 [00:09<00:17, 960641.56it/s]

 39%|█████████████████████████  

 70%|█████████████████████████████████████████████▋                   | 18579456/26421880 [00:16<00:06, 1174261.63it/s]

 71%|██████████████████████████████████████████████                   | 18702336/26421880 [00:17<00:06, 1182679.03it/s]

 71%|██████████████████████████████████████████████▎                  | 18841600/26421880 [00:17<00:06, 1186641.84it/s]

 72%|███████████████████████████████████████████████▎                  | 18964480/26421880 [00:17<00:07, 942871.05it/s]

 73%|███████████████████████████████████████████████▎                 | 19218432/26421880 [00:17<00:06, 1157904.29it/s]

 73%|███████████████████████████████████████████████▋                 | 19365888/26421880 [00:17<00:06, 1164236.25it/s]

 74%|███████████████████████████████████████████████▉                 | 19505152/26421880 [00:17<00:05, 1201334.45it/s]

 74%|████████████████████████████████████████████████▎                | 19644416/26421880 [00:17<00:05, 1236992.60it/s]

 75%|███████████████████████████

Extracting ./data\FashionMNIST\raw\train-images-idx3-ubyte.gz to ./data\FashionMNIST\raw





0it [00:00, ?it/s]


  0%|                                                                                        | 0/29515 [00:00<?, ?it/s]


32768it [00:00, 37317.83it/s]                                                                                          


Extracting ./data\FashionMNIST\raw\train-labels-idx1-ubyte.gz to ./data\FashionMNIST\raw





0it [00:00, ?it/s]


  0%|                                                                                      | 0/4422102 [00:00<?, ?it/s]


  0%|▎                                                                      | 16384/4422102 [00:00<00:55, 79708.12it/s]


  1%|▊                                                                      | 49152/4422102 [00:00<00:44, 97829.93it/s]


  2%|█▋                                                                   | 106496/4422102 [00:00<00:34, 125857.54it/s]


  5%|███▎                                                                 | 212992/4422102 [00:01<00:24, 170370.06it/s]


  6%|████▏                                                                | 270336/4422102 [00:01<00:19, 213320.52it/s]


  8%|█████▊                                                               | 368640/4422102 [00:01<00:14, 277377.10it/s]


 10%|██████▊                                                              | 434176/4422102 [00:01<00:12, 331149.22it/s]




Extracting ./data\FashionMNIST\raw\t10k-images-idx3-ubyte.gz to ./data\FashionMNIST\raw






0it [00:00, ?it/s]



8192it [00:00, 10832.81it/s]                                                                                           


Extracting ./data\FashionMNIST\raw\t10k-labels-idx1-ubyte.gz to ./data\FashionMNIST\raw
Processing...
Done!


## Fully Connected CNN

In [10]:
class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5)
        
        self.fc1 = nn.Linear(in_features=12*4*4, out_features=120)
        self.fc2 = nn.Linear(in_features=120, out_features=60)
        self.out = nn.Linear(in_features=60, out_features=10)
        
    def forward(self, t):
        # (1) input layer
        t = t
        
        # (2) hidden conv layer
        t = self.conv1(t)
        t = F.relu(t)
        t = F.max_pool2d(t, kernel_size=2, stride=2)
        
        # (3) hidden conv layer
        t = self.conv2(t)
        t = F.relu(t)
        t = F.max_pool2d(t, kernel_size=2, stride=2)
        
        # (4) hidden linear layer
        t = t.reshape(-1, 12 * 4 * 4)
        t = self.fc1(t)
        t = F.relu(t)
        
        # (5) hidden linear layer
        t = self.fc2(t)
        t = F.relu(t)
        
        # (6) output layer
        t = self.out(t)
        #t = F.softmax(t, dim=1)
        
        return t

## Training with a Single batch

In [11]:
network = Network()

optimizer = optim.Adam(network.parameters(), lr=0.01)

batch = next(iter(train_loader)) # Get Batch
images, labels = batch 

preds = network(images) # Pass Batch
loss = F.cross_entropy(preds, labels) # Calculate Loss

loss.backward() # Calculate Gradients
optimizer.step() # Update Weights


print('loss 1:', loss.item())
preds = network(images)
loss = F.cross_entropy(preds, labels)
print('loss 2:', loss.item())

loss 1: 2.3202121257781982
loss 2: 2.278287887573242


## Training with multitle ephochs : The Complet etraining loop

In [12]:
network = Network()

optimizer = optim.Adam(network.parameters(), lr=0.01)

epochs = 1

for epoch in range(epochs):

    total_loss = 0
    total_correct = 0

    for batch in train_loader: # Get Batch
        images, labels = batch 

        preds = network(images) # Pass Batch
        loss = F.cross_entropy(preds, labels) # Calculate Loss

        optimizer.zero_grad()
        loss.backward() # Calculate Gradients
        optimizer.step() # Update Weights

        total_loss += loss.item()
        total_correct += get_num_correct(preds, labels)

    print(
        "epoch", epoch, 
        "total_correct:", total_correct, 
        "loss:", total_loss,
        "Loss % ", (total_correct / len(train_set))
    )



26427392it [00:40, 1205321.48it/s]                                                                                     


4423680it [00:25, 780893.44it/s]                                                                                       

epoch 0 total_correct: 47318 loss: 522.4115934371948 Loss %  0.7886333333333333


In [13]:
total_correct / len(train_set)

0.7886333333333333

## Udaicity - Solution2

In [14]:
# transform = tranforms.Compose([tranforms.ToTensor(), tranforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))])
# transform = tranforms.Compose([tranforms.ToTensor()])

# train_set = torchvision.datasets.FashionMNIST(
#     root='./data'
#     ,train=True
#     ,download=True
#     ,transform=transform
# )

# train_loader = torch.utils.data.DataLoader(
#     train_set
#     ,batch_size=64
#     ,shuffle=True
# )

# test_set = torchvision.datasets.FashionMNIST(
#     root='./data'
#     ,train=False
#     ,download=True
#     ,transform=transform
# )

# test_loader = torch.utils.data.DataLoader(
#     test_set
#     ,batch_size=64
#     ,shuffle=True
# )

In [15]:
# class Classifier(nn.Module):
#     def __init__(self):
#         super().__init__()
#         self.fc1 = nn.Linear(784, 256)
#         self.fc2 = nn.Linear(256, 128)
#         self.fc3 = nn.Linear(128,  64)
#         self.fc4 = nn.Linear(64, 10)
        
#         # Dropout moduel with 0.2 drop probability
#         self.dropout = nn.Dropout(p=0.2)
        
#     def forward(self, x):
#         # make sure input tensor is flattented
#         x = x.view(x.shape[0], -1)

#         # Now with dropout
#         x = self.dropout(F.relu(self.fc1(x)))
#         x = self.dropout(F.relu(self.fc2(x)))
#         x = self.dropout(F.relu(self.fc3(x)))
        
#         # Output so no dropout here
#         x = F.log_softmax(self.fc4(x), dim=1)

#         return x
            

In [16]:
# model = Classifier()
# criterion = nn.NLLLoss()
# optimizer = optim.Adam(model.parameters(), lr=0.003)

In [17]:

# epochs = 30
# steps = 0
# train_losses, test_losses = [], []
# for e in range(epochs):
#     running_loss = 0
#     for images, labels in train_loader:
#         logps = model(images)
#         loss = criterion(logps, labels)
        
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()
        
#         running_loss += loss.item()
#     else:
        
#         test_loss = 0
#         accuracy = 0
#         # Trun off gradionts for validation, saves memeory and computations
#         with torch.no_grad():
#             model.eval()
#             for images, labels in test_loader:
#                 log_ps = model(images)
#                 test_loss += criterion(log_ps, labels)
                
#                 ps =  torch.exp(log_ps)
#                 top_p, top_class = ps.topk(1, dim=1)
#                 equals = top_class == labels.view(*top_class.shape)
#                 accuracy += torch.mean(equals.type(torch.FloatTensor))
                
#         model.train()
#         train_losses.append(running_loss/len(train_loader))
#         test_losses.append(test_loss/len(test_loader))
        
#         print("Epoch: {}/{}.. ".format(e+1, epochs),
#               "Training Loss: {:.3f}.. ".format(running_loss/len(train_loader)),
#               "Test Loss: {:.3f}.. ".format(test_loss/len(test_loader)),
#               "Test Accuracy: {:.3f}".format(accuracy/len(test_loader))
#              )

In [18]:
# %matplotlib inline
# %config InlineBackend.figure_format = 'retina'

# import matplotlib.pyplot as plt

# plt.plot(train_losses, label='Training loss')
# plt.plot(test_losses, label='validation loss')
# plt.legend(frameon=False)